# 3 - Run Optimising Sub-pattern Recognition: Robustness to Noise (Generates Data for Figures 11, 12 and 13)

This notebook re-generates data for the noise robustness tests (figures 11, 12, 13) in *Section 9.1: Optimising Sub-pattern Recognition* and *Section 9.2: Explaining the Sub-pattern Optimisations*. 

**To just plot the results:** 
If you would like to just plot the pre-generated paper results, there is no need to re-generate the results as they are cached in `examples/experimental_results/experiment_B_sub_pattern`. In this case, simply use the Jupyter notebook `5 - Experiment_B_sub_pattern - Plot Figs 11 12 and 13.ipynb`.  

**Where the data will be saved:** The results from this test willl be saved in the existing Excel files in the `examples/experimental_results/experiment_B_sub_pattern` folder and overwrite the sheet named `latest_data`. The data generated for the paper is on the `paper_data` sheet, which will not be overwritten. You can select the data to plot (the latest or the paper data) by selecting the correct sheet name in `5 - Experiment_B_sub_pattern - Plot Figs 11 12 and 13.ipynb`. 

This notebook explores the effect on the sub-pattern recognition step through the two optimisation techniques: 

* Optimising the hidden neuron threshold
* Optimising the feature to hidden neuron connections

To examine the effect of the sub-pattern recognition optimisations on ESAM network robustness to noise, the following experiment is used:

<table>
<tr><td>

| Problem B    |          |
| :----------- | :------  | 
| $f$          | 2,000      | 
| $m$          | 1,000 | 
| $s_m$        | 0.1     | 
| $s_n$        | Varying      | 
</td><td>
&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;
</td><td>

| Network B (Baseline) |          |
| :--------------------| :------  | 
| $h$                  | 2        | 
| $\theta$             | 12       | 
| $s^+_{f\_h}$         | 0.1      | 
| $\sigma_{h\_pre+}$   | `FixedProbability` | 
| $s^+_{h\_f}$         | 1     | 
| $s^-_{h\_f}$         | 0 (no inhibition)     | 
| $\sigma_{f\_pre+}$, $\sigma_{f\_pre-}$  |  `FixedProbability`       | 
| $e$                  | 3        | 

</td></tr> </table>

These is based on the baseline experiment presented in [1] (figure 6).

The accuracy of four different networks are compared in the graphs as the number of memories $m$ varies:

* Network A - the baseline network from [1] (as described above).
* Network A with $\theta$ optimised for each test as described in section 6.2.
* Network A with sub-pattern recognition (feature to hidden) connections optimised as described in section 6.3.
* Network A with $\theta$ and the connections optimised

The accuracy of each network is tested with increasing amounts of noise. This is the only variable that changes during the tests.

[1] Hoffmann, H. (2019). Sparse Associative Memory. *Neural Computation*

## Dependencies and problem/network definition

Import the dependencies and define the tests to run. 

### Dependencies

In [1]:
import os
import sys
sys.path.append('../src')
import numpy as np

In [2]:
output_base_dir = '.'+os.sep+'experiment_results'
test_name = 'experiment_B_sub_pattern'

### Define the problem space and base network

Provide the default values for the problem space and the network. The tests will be variations 
on these defaults.

In [3]:
f = 2000
s_m = 200/f
problem_space_default = {'f': f,
                         'm': 1000,
                         's_m': s_m,
                         's_n': 0.1,
                         's_is_prob': False}

network_default = {'f': f,
                   'h': 2,
                   'f_h_sparsity': 0.1,
                   'h_f_sparsity_e': 1,
                   'h_f_sparsity_i': 0,
                   'e': 3,
                   'h_thresh': 12,
                   'cols': 1,
                   'f_h_conn_type': 'FixedProbability',
                   'h_f_conn_type': 'FixedProbability',
                   }


### Define the variable that will change (the x-axis on the final plots)

In this case, the variable that is changed is the amount of noise `s_n`

In [4]:
# Below is the full test, as in the paper.
vary_s_n = np.arange(0, 950, 50)/f

# The full test will take a long time to run. You can experiment with fewer memories.
# You can also reduce the number of simulations that are run per memory variation - see the 
# 'Run the Tests' section below. 
vary_s_n = np.arange(0, 150, 50)/f

### Define the networks to be tested. 

Each network will be tested multiple times for each of the variable options. 

The `problem_vary` and `network_static` dictionary keys indicate that the variation betweeen the tests (for each network) is the problem space - specifically in this case, the amount of noise `s_n`

There are 4 network varieties:
* `original`: the original (base) network from the paper.
* `opt_thresh`: the original network with the hidden neuron threshold optimised for each of the varying memory scenarios.
* `opt_conns`: the original network with the connections optimised.
* `opt_thresh_conns`: the original network combining the optimisations of both `opt_thresh` and `opt_conns`.

In [5]:
import copy

original={
            'problem_vary': problem_space_default.copy(),
            'network_static': network_default.copy(),
            'plot_params':
                {'variable_column': 's_n',
                 'variable_column_name': 'Noise',
                 'variable_is_integer': False}
         }

# Override with the variable parameter. Specifying an array for this parameter will
# indicate to the test runner that there are multiple scenarios to be tested.
original['problem_vary'][original['plot_params']['variable_column']] = vary_s_n

opt_thresh = copy.deepcopy(original)
opt_thresh['network_static']['h_thresh'] = -1     # -1 means optimise the threshold

opt_conns = copy.deepcopy(original)
opt_conns['network_static']['f_h_conn_type'] = 'FixedNumberPre'

opt_thresh_conns = copy.deepcopy(opt_thresh)
opt_thresh_conns['network_static']['f_h_conn_type'] = 'FixedNumberPre'


Put all the network descriptions into a single dictionary for the test runner.

In [6]:
all_tests = {'original': original,
             'opt_thresh': opt_thresh,
             'opt_conns': opt_conns,
             'opt_thresh_conns': opt_thresh_conns
            }

## Run the tests

`num_tests` is the number of networks for each type of test that will be created. Despite having the same network definition, each will vary due to randomness in the data and network connections.

`simulations_per_test` is the number of recalls that will be run on each test network.

The total number of tests that will be run is: `number of test networks * len(vary_s_n) * num_tests * simulations_per_test`

This test is classified as `vary_problem` because the network remains static for each optimisation test, but the problem is varied. 

In [7]:
num_tests = 10
sims_per_test = 10
output_dir = output_base_dir + os.sep + test_name

In [8]:
from simulation_scripts.test_suite_runner import TestSuiteRunner

tsr = TestSuiteRunner(network_default=network_default,
                      problem_space_default=problem_space_default,
                      output_dir=output_dir)


tsr.run_vary_problem_space_test_suite(all_tests = all_tests,
                                      num_tests = num_tests,
                                      sims_per_test = sims_per_test)


Running test set: original defined as: 
    Static network params:    {'f': 2000, 'h': 2, 'f_h_sparsity': 0.1, 'h_f_sparsity_e': 1, 'h_f_sparsity_i': 0, 'e': 3, 'h_thresh': 12, 'cols': 1, 'f_h_conn_type': 'FixedProbability', 'h_f_conn_type': 'FixedProbability'}
    Varying problem space:    {'f': 2000, 'm': 1000, 's_m': 0.1, 's_n': array([0.   , 0.025, 0.05 ]), 's_is_prob': False}

Number of tests per scenario: 10 
Number of simulations per test: 10 

--------------------------------------------------------------------------------------------
Simulation 1 of 10 for test 1 of 3

======= Generating Simulation Data =======
      Number of features (feature neurons): 2000
      Number of memories:                   1000
      Number of simulations:                10
      Signal length:                        200
      Bits to flip for noise:               0
  Learning phase...

=========== ESAM Recall Network =============
    Num features:                   2000
    F to H connection sp

In [9]:
print('Experiment complete, data stored in:\n')
print('  {:80s}    {:10s}\n'.format('File', 'Sheet'))

for test_file, _ in all_tests.items():
    excel_file = output_base_dir + os.sep + test_name + os.sep + test_file + '.xlsx'
    print('  {:80s}    {:10s}'.format(excel_file, 'latest'))

Experiment complete, data stored in:

  File                                                                                Sheet     

  ./experiment_results/experiment_B_sub_pattern/original.xlsx                         latest    
  ./experiment_results/experiment_B_sub_pattern/opt_thresh.xlsx                       latest    
  ./experiment_results/experiment_B_sub_pattern/opt_conns.xlsx                        latest    
  ./experiment_results/experiment_B_sub_pattern/opt_thresh_conns.xlsx                 latest    


## Next steps: Plotting the results

Use the Jupyter notebook `5 - Experiment_B_sub_pattern - Plot Figs 11 12 and 13.ipynb` to plot the results.

You will need to update the `data_sheet` variable to `latest_data` as described in the notebook, else the pre-generated paper data will be plotted.